In [144]:
import pandas as pd
import numpy as np
import os
import datetime as dt

In [117]:
df_grouped_price = pd.read_pickle('{}/nem-data/nemde_clean/df_grouped_price.pkl'.format(os.environ['HOME']))
df_grouped_price = df_grouped_price.reset_index()
df_grouped_price.head(20)

,PeriodID,Price
0,2018-07-31 23:55:00,70.70204
1,2018-07-31 23:50:00,69.24109
2,2018-07-31 23:45:00,69.94850
3,2018-07-31 23:40:00,74.04497
4,2018-07-31 23:35:00,76.44245
5,2018-07-31 23:30:00,72.13931
6,2018-07-31 23:25:00,52.82719
7,2018-07-31 23:20:00,45.83206
8,2018-07-31 23:15:00,47.00597
9,2018-07-31 23:10:00,50.88122


In [143]:
dodgy = df_grouped_price[(df_grouped_price.PeriodID <= df_trading_price_corr[df_trading_price_corr.Summed_dispatch.isna()]["PeriodID"].max()) &
                 (df_grouped_price.PeriodID >= df_trading_price_corr[df_trading_price_corr.Summed_dispatch.isna()]["PeriodID"].min())]
dodgy

,PeriodID,Price
942958,2009-07-14 16:00:00,25.25274
942959,2009-07-14 15:55:00,32.69000
942960,2009-07-14 15:50:00,32.69000
942961,2009-07-14 15:45:00,31.11362
942962,2009-07-14 15:40:00,31.31720
...,...,...
946842,2009-07-01 04:20:00,1.75480
946843,2009-07-01 04:15:00,1.44014
946844,2009-07-01 04:10:00,-0.00006
946845,2009-07-01 04:05:00,-0.00006


In [151]:
dodgy.groupby([dodgy.PeriodID.dt.date]).mean()

,Price
PeriodID,
2009-07-01,20.463798
2009-07-02,27.036960
2009-07-03,25.191636
2009-07-04,28.111905
2009-07-05,31.740115
2009-07-06,34.150252
2009-07-07,33.183796
2009-07-08,36.958959
2009-07-09,31.600491


In [118]:
df_grouped_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946847 entries, 0 to 946846
Data columns (total 2 columns):
PeriodID    946847 non-null datetime64[ns]
Price       946847 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 14.4 MB


In [119]:
print(df_grouped_price.PeriodID.max())
print(df_grouped_price.PeriodID.min())

2018-07-31 23:55:00
2009-07-01 04:00:00


In [120]:
df_trading_price = pd.read_csv('/home/bastian/nem-data/clean/trading_price.csv', index_col=0, parse_dates=True)
df_trading_price.sort_index(ascending=False, inplace=True)
df_trading_price = df_trading_price.loc[:, ["RRP SA1"]]
df_trading_price = df_trading_price[(df_trading_price.index <= df_grouped_price.PeriodID.max()) &
                                   (df_trading_price.index >= df_grouped_price.PeriodID.min())]
df_trading_price = df_trading_price.reset_index()
df_trading_price = df_trading_price.rename(columns={"tempus_interval_time":"PeriodID","RRP SA1":"Trading_price"})

df_trading_price.head()

,PeriodID,Trading_price
0,2018-07-31 23:30:00,72.09
1,2018-07-31 23:00:00,49.86
2,2018-07-31 22:30:00,76.52
3,2018-07-31 22:00:00,81.66
4,2018-07-31 21:30:00,83.25


In [121]:
print(len(df_trading_price))
print(df_trading_price.PeriodID.max())
print(df_trading_price.PeriodID.min())

159256
2018-07-31 23:30:00
2009-07-01 04:00:00


In [123]:
df_trading_dispatch = df_grouped_price.merge(df_trading_price, how='left', on="PeriodID")
df_trading_dispatch = df_trading_dispatch.rename(columns={"Price":"Dispatch_price","RRP SA1":"Trading_price"})
df_trading_dispatch.head(20)
print(sum(df_trading_dispatch.Trading_price.isna()))
df_trading_dispatch.head()
#df_trading_dispatch = df_trading_dispatch[]

789043


,PeriodID,Dispatch_price,Trading_price
0,2018-07-31 23:55:00,70.70204,NaN
1,2018-07-31 23:50:00,69.24109,NaN
2,2018-07-31 23:45:00,69.94850,NaN
3,2018-07-31 23:40:00,74.04497,NaN
4,2018-07-31 23:35:00,76.44245,NaN


In [280]:
df_trading_price_corr = df_trading_dispatch[~df_trading_dispatch.Trading_price.isna()].drop(columns="Dispatch_price").reset_index(drop=True)
df_trading_price_corr.iloc[1479:1484]

,PeriodID,Trading_price
1479,2018-07-01 04:00:00,78.96
1480,2018-06-01 03:30:00,55.00
1481,2018-06-01 03:00:00,55.66
1482,2018-06-01 02:30:00,57.77
1483,2018-06-01 02:00:00,61.54


In [188]:
count = 0
for i,j in df_trading_price_corr.PeriodID[1:].iteritems():
    count += 1
    delta = abs(df_trading_price_corr.PeriodID[i] - df_trading_price_corr.PeriodID[i-1])
    print(i,delta.total_seconds())
    if count > 5:
        break

1 1800.0
2 1800.0
3 1800.0
4 1800.0
5 1800.0
6 1800.0


In [277]:
def correct_timedelta(table,col,time_diff):
    '''
    table=table_of_interest
    col="column_of_interest"
    time_diff=time_diff in seconds as int
    '''
    lst = []
    lst_i = []
    for i,j in table[col][1:].iteritems():
        delta = abs(table[col][i] - table[col][i-1])
        if int(delta.total_seconds()) != int(time_diff):
            lst.append(("from index {} on, it has been {} s or {} h.".format(i-1,int(delta.total_seconds()),(int(delta.total_seconds()/3600)))))
            lst_i.append((i-1,int(delta.total_seconds())))
    return lst, lst_i

In [278]:
check, indices = correct_timedelta(df_trading_price_corr,"PeriodID",1800)

In [279]:
print(indices)
check

[(1479, 2593800), (28863, 3600), (29968, 3600), (29973, 3600), (29976, 3600), (32732, 3600), (32875, 3600), (42737, 3600), (85886, 3600), (144752, 3600), (154733, 3600), (155426, 3600), (155548, 3600)]


['from index 1479 on, it has been 2593800 s or 720 h.',
 'from index 28863 on, it has been 3600 s or 1 h.',
 'from index 29968 on, it has been 3600 s or 1 h.',
 'from index 29973 on, it has been 3600 s or 1 h.',
 'from index 29976 on, it has been 3600 s or 1 h.',
 'from index 32732 on, it has been 3600 s or 1 h.',
 'from index 32875 on, it has been 3600 s or 1 h.',
 'from index 42737 on, it has been 3600 s or 1 h.',
 'from index 85886 on, it has been 3600 s or 1 h.',
 'from index 144752 on, it has been 3600 s or 1 h.',
 'from index 154733 on, it has been 3600 s or 1 h.',
 'from index 155426 on, it has been 3600 s or 1 h.',
 'from index 155548 on, it has been 3600 s or 1 h.']

In [281]:
check, indices = correct_timedelta(df_grouped_price,"PeriodID",300)

In [285]:
print(indices)
check

[(8879, 2592300), (23741, 600), (28750, 600), (75753, 600), (76916, 600), (173184, 600), (179819, 600), (179831, 600), (179849, 600), (179852, 600), (179858, 600), (179870, 600), (179873, 1200), (179892, 600), (190312, 600), (190316, 600), (190362, 600), (196113, 600), (196404, 900), (196405, 600), (197229, 600), (197232, 600), (197266, 600), (201290, 600), (239428, 600), (256441, 600), (490105, 600), (515339, 1500), (793820, 600), (794497, 600), (818962, 600), (868534, 600), (887404, 600), (925535, 600), (925538, 600), (928129, 600), (928146, 600), (928152, 600), (928416, 600), (928418, 600), (928697, 600), (932580, 600), (933317, 600), (933323, 600)]


['from index 8879 on, it has been 2592300 s or 720 h.',
 'from index 23741 on, it has been 600 s or 0 h.',
 'from index 28750 on, it has been 600 s or 0 h.',
 'from index 75753 on, it has been 600 s or 0 h.',
 'from index 76916 on, it has been 600 s or 0 h.',
 'from index 173184 on, it has been 600 s or 0 h.',
 'from index 179819 on, it has been 600 s or 0 h.',
 'from index 179831 on, it has been 600 s or 0 h.',
 'from index 179849 on, it has been 600 s or 0 h.',
 'from index 179852 on, it has been 600 s or 0 h.',
 'from index 179858 on, it has been 600 s or 0 h.',
 'from index 179870 on, it has been 600 s or 0 h.',
 'from index 179873 on, it has been 1200 s or 0 h.',
 'from index 179892 on, it has been 600 s or 0 h.',
 'from index 190312 on, it has been 600 s or 0 h.',
 'from index 190316 on, it has been 600 s or 0 h.',
 'from index 190362 on, it has been 600 s or 0 h.',
 'from index 196113 on, it has been 600 s or 0 h.',
 'from index 196404 on, it has been 900 s or 0 h.',
 'from inde

In [240]:
count=0
for i in check:
    if i != 1800:
        count+=1
print(count)

13


In [236]:
myset = set(check)
print(myset)

{1800, 2593800, 3600}


In [178]:
df_trading_price_corr.loc[:,"Trading_price"].iloc[i]

72.09

In [163]:
d = df_trading_price_corr.PeriodID[0] - df_trading_price_corr.PeriodID[1]
d.total_seconds()

1800.0

In [131]:
print(df_trading_price_corr.PeriodID.max())
print(df_trading_price_corr.PeriodID.min())

2018-07-31 23:30:00
2009-07-01 04:00:00


In [132]:
df_trading_price_corr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157804 entries, 0 to 157803
Data columns (total 2 columns):
PeriodID         157804 non-null datetime64[ns]
Trading_price    157804 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.4 MB


In [174]:
df_trading_dispatch.loc[:,["Dispatch_price"]].iloc[:5,:].agg(['mean', 'max','min'])

,Dispatch_price
mean,72.07581
max,76.44245
min,69.24109


In [153]:
bin_column = df_grouped_price.loc[:,"Price"]
bin_size = 6
bin_sums = []
#count=0
for i in df_grouped_price.index[:df_grouped_price.shape[0]:bin_size]:
    #count+=1
    j = i + bin_size
    bin_sum = sum(bin_column.iloc[i:j])/6
    #print(i,j)
    #print(bin_column.iloc[i:j])
    #print(sum(bin_column.iloc[i:j]))
    #print(sum(bin_column.iloc[i:j])/6)
    bin_sums.append(bin_sum)
    #if count > 5:
    #    break

In [154]:
len(bin_sums) #make assert bin_sums == bla / 6 == len(df_trading_price)

157808

In [155]:
df_trading_price_corr["Summed_dispatch"] = pd.DataFrame(data=bin_sums)
print(len(df_trading_price_corr[df_trading_price_corr.Summed_dispatch.isna()]))
print(df_trading_price_corr[df_trading_price_corr.Summed_dispatch.isna()]["PeriodID"].max())
print(df_trading_price_corr[df_trading_price_corr.Summed_dispatch.isna()]["PeriodID"].min())
df_trading_price_corr.head()

#assert!!!

0
NaT
NaT


,PeriodID,Trading_price,Summed_dispatch
0,2018-07-31 23:30:00,72.09,72.086393
1,2018-07-31 23:00:00,49.86,49.860560
2,2018-07-31 22:30:00,76.52,76.524205
3,2018-07-31 22:00:00,81.66,81.655690
4,2018-07-31 21:30:00,83.25,83.248513


In [60]:
bla = []
for i in pommes.index[0:33:6]:
    j = i + 5
    print(i,j)
    k = sum(df_trading_dispatch.loc[:,"Dispatch_price"].iloc[i:j])
    print(k)
    bla.append(k)

0 5
360.37905
6 11
248.11188999999996
12 17
380.45523
18 23
407.37278999999995
24 29
414.49107000000004


In [12]:
bucket_size = 5
count = 0
j_sum = 0
for i,j in pommes.iteritems():
    count += 1
    for k in range
    j_sum += j
    print(j_sum)

1
70.70204
70.70204
2
69.24109
139.94313
3
69.9485
209.89163
4
74.04497
283.9366
5
76.44245
360.37905
6
72.13931000000001
432.51836000000003
7
52.82718999999998
485.34555
8
45.832060000000006
531.17761
9
47.00597
578.18358
10
50.88122
629.0648


In [42]:
df_trading_dispatch.index[4]

Timestamp('2018-07-31 21:30:00')

In [51]:
file = '{}/nem-data/nemde_clean/concat_for_energy_SA1'.format(os.environ['HOME'])
meh = pd.read_pickle(file)

dtype('O')

In [54]:
meh.set_index("PeriodID", inplace=True)
meh.index = pd.to_datetime(meh.index)

In [60]:
import datetime as dt
import pytz
meh.index.tz_convert(tz=pytz.FixedOffset(300))

DatetimeIndex(['2015-06-23 08:40:00+05:00', '2015-06-23 08:40:00+05:00',
               '2015-06-23 08:40:00+05:00', '2015-06-23 08:40:00+05:00',
               '2015-06-23 10:00:00+05:00', '2015-06-23 10:00:00+05:00',
               '2015-06-23 10:00:00+05:00', '2015-06-23 10:00:00+05:00',
               '2015-06-23 10:00:00+05:00', '2015-06-23 10:00:00+05:00',
               ...
               '2012-01-11 20:25:00+05:00', '2012-01-11 20:25:00+05:00',
               '2012-01-11 20:25:00+05:00', '2012-01-11 20:25:00+05:00',
               '2012-01-11 07:45:00+05:00', '2012-01-11 07:45:00+05:00',
               '2012-01-11 07:45:00+05:00', '2012-01-11 07:45:00+05:00',
               '2012-01-11 07:45:00+05:00', '2012-01-11 07:45:00+05:00'],
              dtype='datetime64[ns, pytz.FixedOffset(300)]', name='PeriodID', length=5951880, freq=None)